# AIG 230 – Week 2 Lab
## From Raw Text to Corpus: Tokenization, Normalization, and Vocabulary

Industry Context: Exploring the State of the Union Corpus

## Learning Objectives
- Understand raw text, documents, and corpora
- Explore a real-world corpus
- Compare NLTK and spaCy preprocessing pipelines
- Perform tokenization, normalization, and vocabulary analysis

In [2]:

import nltk
import spacy
import string
from collections import Counter

nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("state_union")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package state_union to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package state_union is already up-to-date!


True

In [3]:
# Load spaCy English model
# Run once if needed:
!python -m spacy download en_core_web_sm

nlp = spacy.load("en_core_web_sm")

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --- ------------------------------------ 1.0/12.8 MB 16.7 MB/s eta 0:00:01
     ----------------------- ---------------- 7.6/12.8 MB 29.4 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 32.2 MB/s  0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


Breaking it down:

```
spacy.load() - loads a pre-trained language model
```

"en_core_web_sm" - the small English model that includes:

- Tokenizer - splits text into words/sentences
- Part-of-speech tagger - identifies noun, verb, adjective, etc.
- Dependency parser - analyzes grammatical relationships
- Named entity recognizer - identifies people, places, organizations
- Word vectors - semantic meaning of words

nlp = - stores the loaded model in a variable so you can use it to process text

## Part 1 – Obtaining the Corpus

In [4]:

from nltk.corpus import state_union
state_union.fileids()[:10]


['1945-Truman.txt',
 '1946-Truman.txt',
 '1947-Truman.txt',
 '1948-Truman.txt',
 '1949-Truman.txt',
 '1950-Truman.txt',
 '1951-Truman.txt',
 '1953-Eisenhower.txt',
 '1954-Eisenhower.txt',
 '1955-Eisenhower.txt']

Each file is a document. The collection is the corpus.

In [5]:

len(state_union.fileids())


65

## Part 2 – Inspect Raw Text

In [6]:

sample_file = state_union.fileids()[0]
raw_text = state_union.raw(sample_file)
raw_text[:1000]


"PRESIDENT HARRY S. TRUMAN'S ADDRESS BEFORE A JOINT SESSION OF THE CONGRESS\n \nApril 16, 1945\n\nMr. Speaker, Mr. President, Members of the Congress:\nIt is with a heavy heart that I stand before you, my friends and colleagues, in the Congress of the United States.\nOnly yesterday, we laid to rest the mortal remains of our beloved President, Franklin Delano Roosevelt. At a time like this, words are inadequate. The most eloquent tribute would be a reverent silence.\nYet, in this decisive hour, when world events are moving so rapidly, our silence might be misunderstood and might give comfort to our enemies.\nIn His infinite wisdom, Almighty God has seen fit to take from us a great man who loved, and was beloved by, all humanity.\nNo man could possibly fill the tremendous void left by the passing of that noble soul. No words can ease the aching hearts of untold millions of every race, creed and color. The world knows it has lost a heroic champion of justice and freedom.\nTragic fate has 

## Part 3 – Word Tokenization

📌 Tokenization splits text into meaningful units (tokens).
There is no universal standard, but conventions vary by task and language.

## Part 4 – Sentence Tokenization

## Part 5 – Normalization
Normalization makes text more consistent.

## Part 6 – Stop Word Removal

📌 Stop words are common words that often add little semantic meaning.

## Part 7 – Vocabulary and Frequency

📌 The vocabulary is the set of unique tokens in a corpus.

# Stemming vs Lemmatization

Stemming and lemmatization are both normalization techniques, but they make very different trade-offs.

Stemming is fast and rule-based but can distort meaning

Lemmatization is slower but linguistically informed

In industry, the choice depends on task, domain, and interpretability requirements.

“democracy” → “democraci”

stems are not necessarily real words

### spaCy does not include a built-in stemmer by default.

This is not a limitation. It is a design choice.

spaCy prioritizes:

- linguistically informed processing

- lemmatization over stemming

However, in real pipelines, you can still perform stemming alongside spaCy.

NLTK does support lemmatization, but it requires:

- a lemmatizer

- part-of-speech information to work well

By default, NLTK’s lemmatizer assumes nouns

Notice that many verbs are not lemmatized correctly.
This is because NLTK’s lemmatizer defaults to noun POS tags.

# From Words to Subwords: Byte Pair Encoding (BPE)

So far, we have treated words as the basic unit of meaning.
Modern NLP systems often go one step further and operate on subword units.

One of the most common subword tokenization methods is Byte Pair Encoding (BPE).

In large corpora like the State of the Union addresses, word-level tokenization creates several problems:

Rare words appear very infrequently

New words appear over time (e.g. cybersecurity, biotechnology)

Related words are treated as completely separate tokens

Subword tokenization solves this by breaking words into frequently occurring pieces.

In [33]:
# We will use a small subset of real policy-related words that appear in State of the Union speeches.

words = [
    "democracy",
    "democratic",
    "democratization",
    "economy",
    "economic",
    "economics"
]

words
# At the word level, all of these are treated as separate tokens.

['democracy',
 'democratic',
 'democratization',
 'economy',
 'economic',
 'economics']

## Step 1 – Character-Level Representation

BPE starts by representing each word as a sequence of characters
(with a special end-of-word marker).

In [34]:
char_tokens = [list(word) + ["</w>"] for word in words]
char_tokens


[['d', 'e', 'm', 'o', 'c', 'r', 'a', 'c', 'y', '</w>'],
 ['d', 'e', 'm', 'o', 'c', 'r', 'a', 't', 'i', 'c', '</w>'],
 ['d',
  'e',
  'm',
  'o',
  'c',
  'r',
  'a',
  't',
  'i',
  'z',
  'a',
  't',
  'i',
  'o',
  'n',
  '</w>'],
 ['e', 'c', 'o', 'n', 'o', 'm', 'y', '</w>'],
 ['e', 'c', 'o', 'n', 'o', 'm', 'i', 'c', '</w>'],
 ['e', 'c', 'o', 'n', 'o', 'm', 'i', 'c', 's', '</w>']]

## Step 2 – Count Frequent Character Pairs

BPE repeatedly merges the most frequent adjacent character pairs across the corpus.

In [35]:
from collections import Counter

pair_counts = Counter()

for word in char_tokens:
    for i in range(len(word) - 1):
        pair = (word[i], word[i+1])
        pair_counts[pair] += 1

pair_counts.most_common(10)


[(('o', 'n'), 4),
 (('d', 'e'), 3),
 (('e', 'm'), 3),
 (('m', 'o'), 3),
 (('o', 'c'), 3),
 (('c', 'r'), 3),
 (('r', 'a'), 3),
 (('a', 't'), 3),
 (('t', 'i'), 3),
 (('i', 'c'), 3)]

## Step 3 – Merge Frequent Pairs (Conceptual)

The most frequent pair is ('o', 'n').
BPE merges it into a new token: "on".

This process repeats many times, gradually forming meaningful subwords.

In [36]:
bpe_tokens_example = [
    ["democr", "acy</w>"],
    ["democr", "atic</w>"],
    ["democr", "atization</w>"],
    ["econ", "omy</w>"],
    ["econ", "omic</w>"],
    ["econ", "omics</w>"]
]

bpe_tokens_example


[['democr', 'acy</w>'],
 ['democr', 'atic</w>'],
 ['democr', 'atization</w>'],
 ['econ', 'omy</w>'],
 ['econ', 'omic</w>'],
 ['econ', 'omics</w>']]